## Tuning YAHPO using `HPBandSter`

In order to demonstrate using YAHPO Gym more in-depth we provide a full example benchmarking `HPBandSter` on an `lcbench` task.

Note, that this requires the `hpbandster` module to be installed.

We again start by importing the relevant modules:

In [1]:
from yahpo_gym import *
import time
import numpy as np

First, we define a worker class as required by `HPBandSter` that internally calls our `objective_function`.

In [2]:
from hpbandster.core.worker import Worker
import hpbandster.core.nameserver as hpns
from hpbandster.optimizers import BOHB as BOHB

class lcbench(Worker):

    def __init__(self, *args, sleep_interval=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.bench = bench
        self.sleep_interval = sleep_interval

    def compute(self, config, budget, **kwargs):
        """
        Args:
            config: dictionary containing the sampled configurations by the optimizer
            budget: (float) amount of epochs the model can use to train

        Returns:
            dictionary with mandatory fields:
                "loss" (scalar)
                "info" (dict)
        """

        config.update({"epoch": int(np.round(budget))})  # update epoch
        result = bench.objective_function(config)  # evaluate

        time.sleep(self.sleep_interval)

        return({
                    "loss": - float(result[0].get("val_accuracy")),  # we want to maximize validation accuracy
                    "info": "empty"
                })
    
    @staticmethod
    def get_configspace():
        # remove the epoch fidelity parameter for HPBandSter
        cs = bench.get_opt_space(drop_fidelity_params = True)
        return(cs)

Using this worker class, we can now run the full benchmark:

In [3]:
# Initialize the set providing a scenario
bench = benchmark_set.BenchmarkSet("lcbench", instance = "3945")

# Start up the nameserver
NS = hpns.NameServer(run_id="lcbench", host="127.0.0.1", port=None)
NS.start()

# Run BOHB
w = lcbench(sleep_interval=0, nameserver="127.0.0.1", run_id ="lcbench")
w.run(background=True)

bohb = BOHB(configspace=w.get_configspace(),
            run_id="lcbench", nameserver="127.0.0.1",
            min_budget=1, max_budget=52)

res = bohb.run(n_iterations=1)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

20:49:35 wait_for_workers trying to get the condition
20:49:35 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7f4e9db91af0; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
20:49:35 DISPATCHER: started the 'discover_worker' thread
20:49:35 WORKER: No dispatcher found. Waiting for one to initiate contact.
20:49:35 WORKER: start listening for jobs
20:49:35 DISPATCHER: started the 'job_runner' thread
20:49:35 DISPATCHER: Pyro daemon running on localhost:43081
20:49:35 DISPATCHER: Starting worker discovery
20:49:35 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
20:49:35 DISPATCHER: discovered new worker, hpbandster.run_lcbench.worker.flo-x380.13581139977482281344
20:49:35 HBMASTER: number of workers changed to 1
20:49:35 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
20:49:35 HBMASTER: only 1 worker(s) available, waiting for at least 1.
20:49:35 adjust_queue_size: lock accquired
20:49:35 HBMASTER: adjusted queue size to (0, 1)
20:49:

Then we obtain the final results:

In [4]:
id2config = res.get_id2config_mapping()
id2config

{(0,
  0,
  0): {'config': {'OpenML_task_id': '3945',
   'batch_size': 475,
   'learning_rate': 0.000279421845934996,
   'max_dropout': 0.23277220802293364,
   'max_units': 564.7617170743333,
   'momentum': 0.28984493464511285,
   'num_layers': 3,
   'weight_decay': 0.08149013793334492}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  1): {'config': {'OpenML_task_id': '3945',
   'batch_size': 56,
   'learning_rate': 0.006354512231197386,
   'max_dropout': 0.53899351576789,
   'max_units': 760.0387687665543,
   'momentum': 0.9524552152326227,
   'num_layers': 1,
   'weight_decay': 0.014299983728636233}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  2): {'config': {'OpenML_task_id': '3945',
   'batch_size': 470,
   'learning_rate': 0.054473558440077924,
   'max_dropout': 0.7775565406704537,
   'max_units': 877.9389562040402,
   'momentum': 0.20871034235326025,
   'num_layers': 1,
   'weight_decay': 0.09301422668185681}, 'config_info': {'model_based_pick': False}},
 (0

In [5]:
incumbent = res.get_incumbent_id()
incumbent

(0, 0, 22)

In [6]:

print("Best found configuration:", id2config[incumbent]["config"])
print("A total of %i unique configurations where sampled." % len(id2config.keys()))
print("A total of %i runs where executed." % len(res.get_all_runs()))
print("Total budget corresponds to %.1f full function evaluations."%(sum([r.budget for r in res.get_all_runs()])/1))

Best found configuration: {'OpenML_task_id': '3945', 'batch_size': 34, 'learning_rate': 0.020516887229563185, 'max_dropout': 0.2631625196046725, 'max_units': 274.99575120920446, 'momentum': 0.9868245594684214, 'num_layers': 5, 'weight_decay': 0.00683309415048856}
A total of 27 unique configurations where sampled.
A total of 40 runs where executed.
Total budget corresponds to 208.0 full function evaluations.
